In [ ]:
import os

import mlflow
import faiss
import torch
import pandas as pd

from config import CONFIG

In [ ]:
class SimilarItemSearch:
    
    def __init__(self, vectors):
        vs = vectors.shape[1] # vector size
        self.item_size = vectors.shape[0] # vector size
        self.index = faiss.IndexFlatL2(vs)
        self.index.add(vectors)
    
    def get_item(self, item_id, k):
        if item_id >= self.item_size :
            raise IndexError(f' item_id : {item_id}, item_size : {self.item_size}')
        query_vector = index.reconstruct(item_id).reshape(1,-1)
        
        dist, idx = index.search(query_vector, k=k+1)
        idx = idx[0][1:]
        return idx 

In [ ]:
model = mlflow.pytorch.load_model('runs:/7b2ecfc6d6a24e15b11c3a094c60a90f/torch_model')
param = model.get_vector()
sim_item_search = SimilarItemSearch(param)

In [ ]:
save_dir = os.path.join(CONFIG.DATA, '1M')
train_data = pd.read_csv(os.path.join(save_dir, 'train.tsv'), sep='\t')
test_data = pd.read_csv(os.path.join(save_dir, 'test.tsv'), sep='\t')
item_meta = pd.read_csv(os.path.join(save_dir, 'item_meta.tsv'), sep='\t', low_memory=False)
# user_meta = pd.read_csv(os.path.join(save_dir, 'user_meta.tsv'), sep='\t')

In [ ]:
recommendation = sim_item_search.get_item(1600, k=50)
item_meta[item_meta.item_id == 1600]

In [ ]:
item_meta[item_meta.item_id == 47]

In [ ]:
pd.concat([item_meta[item_meta.item_id == r] for r in recommendation])

In [ ]:
recom_ids = []
target_ids = []
for iid, pid in zip(test_data.item_id, test_data.prev_item_id):
    target_ids.append(iid)
    recom_ids.append(
        list(sim_item_search.get_item(pid, k=50))
    )

In [ ]:
from model.metrics import RecallAtK

In [ ]:
racall_func = RecallAtK()

In [ ]:
str(racall_func), racall_func(recom_ids, target_ids)